<a href="https://colab.research.google.com/github/krishna-kenny/nbaWinNeuralNetModel/blob/main/nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.4/279.4 kB 7.1 MB/s eta 0:00:00


In [2]:
import time
import pandas as pd
from nba_api.stats.endpoints import TeamInfoCommon, TeamGameLogs, PlayerGameLogs, LeagueGameFinder, LeagueLeaders, PlayerCareerStats
from nba_api.stats.static import teams

# Maximum number of retries for each API call
MAX_RETRIES = 3

def fetch_with_retries(func, *args, **kwargs):
    """Attempts a function call up to MAX_RETRIES with exponential backoff."""
    for attempt in range(MAX_RETRIES):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            wait_time = 2**attempt  # Exponential backoff
            print(f"Error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print(f"Failed after {MAX_RETRIES} attempts.")
    return None

def get_team_info(seasons):
    """Fetches relevant team information for the specified seasons."""
    print("Fetching team information...")
    nba_teams = teams.get_teams()
    team_data = []

    for team in nba_teams:
        team_info = fetch_with_retries(
            TeamInfoCommon,
            team_id=team["id"],
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if team_info:
            df_team = team_info.get_data_frames()[0]
            df_team = df_team[["TEAM_ID", "TEAM_ABBREVIATION"]]  # Only keep relevant features
            team_data.append(df_team)
            time.sleep(0.6)  # Delay to avoid API rate limits

    if team_data:
        df_teams = pd.concat(team_data, ignore_index=True)
        df_teams.to_csv("nba_team_data.csv", index=False)
    else:
        print("No team data fetched.")

def get_team_game_logs(seasons):
    """Fetches team game logs for the specified seasons and processes the MATCHUP column."""
    print("Fetching team game logs...")
    game_log_data = []

    for season in seasons:
        game_logs = fetch_with_retries(
            TeamGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if game_logs:
            df_game_logs = game_logs.get_data_frames()[0]
            # Keep only relevant columns
            df_game_logs = df_game_logs[[
                "GAME_ID", "GAME_DATE", "MATCHUP", "WL"
            ]]
            game_log_data.append(df_game_logs)
            time.sleep(0.6)  # Delay to respect rate limits

    if game_log_data:
        # Concatenate all game logs
        df_all_game_logs = pd.concat(game_log_data, ignore_index=True)

        # Process MATCHUP column to create team1 and team2 columns
        matchups_split = df_all_game_logs['MATCHUP'].str.split(' @ | vs. ', expand=True)
        df_all_game_logs['TEAM1'] = matchups_split[0]
        df_all_game_logs['TEAM2'] = matchups_split[1]

        # Drop the original MATCHUP column if no longer needed
        df_all_game_logs.drop(columns=['MATCHUP'], inplace=True)

        # Save the processed DataFrame to a CSV file
        df_all_game_logs.to_csv("nba_game_logs.csv", index=False)
        print("Processed game logs saved to 'nba_game_logs.csv'.")
    else:
        print("No game log data fetched.")



def get_player_game_logs(seasons):
    """Fetches player game logs for the specified seasons."""
    print("Fetching player game logs...")
    player_game_log_data = []

    for season in seasons:
        player_game_logs = fetch_with_retries(
            PlayerGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if player_game_logs:
            df_player_game_logs = player_game_logs.get_data_frames()[0]
            # Keep only relevant columns
            df_player_game_logs = df_player_game_logs[[
                "SEASON_YEAR", "GAME_ID", "TEAM_ID", "PLAYER_ID", "PLAYER_NAME", "PTS", "REB", "AST", "STL", "BLK",
                "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", "TOV", "PF"
            ]]
            player_game_log_data.append(df_player_game_logs)
            time.sleep(0.6)

    if player_game_log_data:
        df_all_player_game_logs = pd.concat(player_game_log_data, ignore_index=True)
        df_all_player_game_logs.to_csv("nba_player_game_logs.csv", index=False)
    else:
        print("No player game log data fetched.")

def get_league_game_data():
    """Fetches league-wide game data with relevant features for a neural network."""
    print("Fetching league game data for NN...")
    game_data = fetch_with_retries(LeagueGameFinder, timeout=60)
    if game_data:
        df_game_data = game_data.get_data_frames()[0]
        # Relevant columns for neural network input
        relevant_columns = [
            "SEASON_ID", "TEAM_ID", "TEAM_ABBREVIATION", "TEAM_NAME", "GAME_ID",
            "GAME_DATE", "MATCHUP", "WL", "MIN", "PTS", "FGM", "FGA", "FG_PCT",
            "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB",
            "REB", "AST", "STL", "BLK", "TOV", "PF", "PLUS_MINUS"
        ]
        df_nn_data = df_game_data[relevant_columns]
        df_nn_data.to_csv("nba_league_game.csv", index=False)
    else:
        print("No league game data fetched.")


def get_league_leaders():
    """Fetches league leaders data with relevant columns for analysis."""
    print("Fetching league leaders data...")
    leaders_data = fetch_with_retries(LeagueLeaders, timeout=60)
    if leaders_data:
        df_leaders = leaders_data.get_data_frames()[0]
        # Select only relevant columns
        relevant_columns = [
            "PLAYER_ID", "PLAYER", "TEAM_ID", "TEAM", "GP", "MIN", "FGM", "FGA",
            "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB",
            "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "EFF"
        ]
        df_relevant_leaders = df_leaders[relevant_columns]
        df_relevant_leaders.to_csv("nba_league_leaders_relevant.csv", index=False)
    else:
        print("No league leaders data fetched.")


def get_player_career_stats():
    """Fetches career stats for players."""
    print("Fetching player career stats...")
    career_stats_data = []
    nba_teams = teams.get_teams()
    for team in nba_teams:
        players = team.get("players", [])
        for player in players:
            career_stats = fetch_with_retries(PlayerCareerStats, player_id=player["id"], timeout=60)
            if career_stats:
                df_career_stats = career_stats.get_data_frames()[0]
                # Keep only relevant columns
                df_career_stats = df_career_stats[[
                    "PLAYER_ID", "PLAYER_NAME", "GP", "PTS", "REB", "AST", "FG_PCT", "FG3_PCT", "FT_PCT"
                ]]
                career_stats_data.append(df_career_stats)
                time.sleep(0.6)

    if career_stats_data:
        df_all_career_stats = pd.concat(career_stats_data, ignore_index=True)
        df_all_career_stats.to_csv("nba_player_career_stats.csv", index=False)
    else:
        print("No player career stats data fetched.")


# Define the list of seasons
seasons = ["2023-24", "2024-25"]

# Run functions to save data to CSV files
get_team_info(seasons)
print("Team information data stored.")

get_team_game_logs(seasons)
print("Team game logs data stored.")

get_player_game_logs(seasons)
print("Player game logs data stored.")

get_league_game_data()
print("League game data stored.")

get_league_leaders()
print("League leaders data stored.")

get_player_career_stats()
print("Player career stats data stored.")


Fetching team information...
Team information data stored.
Fetching team game logs...
Processed game logs saved to 'nba_game_logs.csv'.
Team game logs data stored.
Fetching player game logs...
Player game logs data stored.
Fetching league game data for NN...
League game data stored.
Fetching league leaders data...
League leaders data stored.
Fetching player career stats...
No player career stats data fetched.
Player career stats data stored.


In [3]:
import pandas as pd

# Load player game logs from the CSV file
file_path = "nba_player_game_logs.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Load league leaders data (assumed to have 'PLAYER_ID' column)
league_leaders_path = "nba_league_leaders_relevant.csv"  # Update with your actual file path
league_leaders_df = pd.read_csv(league_leaders_path)

# Exclude non-numerical columns explicitly
no_aggregate_columns = ['PLAYER_ID', 'SEASON_YEAR', 'PLAYER_NAME', 'TEAM_ID']  # Adjust as necessary
numerical_columns = [col for col in df.columns if col not in no_aggregate_columns]

# Group by PLAYER_ID and SEASON_YEAR
grouped = df.groupby(['PLAYER_ID', 'SEASON_YEAR'])

# Aggregate numerical columns using mean and count the number of games
aggregated_data = grouped[numerical_columns].mean().reset_index()

# Add non-numerical columns using the first value in the group (like TEAM_ID)
aggregated_data['TEAM_ID'] = grouped['TEAM_ID'].first().values

# Add games played as a new column
aggregated_data['GAMES_PLAYED'] = grouped.size().values

# Mark league leaders (ignoring SEASON_YEAR)
league_leader_set = set(league_leaders_df['PLAYER_ID'])

# Add a column to indicate whether the player is a league leader
aggregated_data['LEAGUE_LEADER'] = aggregated_data['PLAYER_ID'].apply(
    lambda player_id: 1 if player_id in league_leader_set else 0
)

# Save the aggregated data for further use
output_path = "nba_player_aggregated_data.csv"
aggregated_data.to_csv(output_path, index=False)

print(f"Aggregated data saved to '{output_path}'.")



Aggregated data saved to 'nba_player_aggregated_data.csv'.


In [4]:
import pandas as pd

# Load aggregated player data
player_aggregated_file = "nba_player_aggregated_data.csv"  # Update with your actual file path
player_df = pd.read_csv(player_aggregated_file)

# Define non-numerical columns to exclude
no_aggregate_columns = ['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'SEASON_YEAR']
numerical_columns = [col for col in player_df.columns if col not in no_aggregate_columns]

# Multiply each player's stats by their 'MIN' to weight the statistics
for col in numerical_columns:
    player_df[f"{col}_WEIGHTED"] = player_df[col] * player_df['MIN']

# Group by TEAM_ID and SEASON_YEAR
grouped = player_df.groupby(['TEAM_ID', 'SEASON_YEAR'])

# Compute team-level weighted stats as the sum of weighted stats divided by the total 'MIN'
team_aggregated_data = grouped[[f"{col}_WEIGHTED" for col in numerical_columns]].sum()
team_aggregated_data.columns = numerical_columns  # Rename back to original column names

# Compute total minutes played by the team
team_aggregated_data['TOTAL_MIN'] = grouped['MIN'].sum()

# Normalize weighted stats by dividing by TOTAL_MIN
for col in numerical_columns:
    team_aggregated_data[col] = team_aggregated_data[col] / team_aggregated_data['TOTAL_MIN']

# Add additional columns
team_aggregated_data['TEAM_GAMES_PLAYED'] = grouped['GAMES_PLAYED'].sum()  # Total games played by players in the team

# Reset index to flatten the DataFrame
team_aggregated_data.reset_index(inplace=True)

# Save the aggregated data for further use
output_path = "nba_team_aggregated_data.csv"
team_aggregated_data.to_csv(output_path, index=False)

print(f"Team aggregated data saved to '{output_path}'.")
print(team_aggregated_data.head())


Team aggregated data saved to 'nba_team_aggregated_data.csv'.
      TEAM_ID SEASON_YEAR       GAME_ID        PTS       REB       AST  \
0  1610612737     2023-24  2.230063e+07  12.870024  4.654347  3.075042   
1  1610612737     2024-25  2.240027e+07  12.356253  4.594481  3.240124   
2  1610612738     2023-24  2.230062e+07  12.808638  4.787837  2.814556   
3  1610612738     2024-25  2.240028e+07  14.044964  4.972278  2.860136   
4  1610612739     2023-24  2.230061e+07  12.277182  4.469090  3.097155   

        STL       BLK        MIN    FG_PCT   FG3_PCT    FT_PCT       TOV  \
0  0.815135  0.460030  25.954890  0.442187  0.280925  0.491170  1.439518   
1  1.069110  0.521717  25.046542  0.444820  0.285614  0.486928  1.727768   
2  0.749032  0.717085  24.908992  0.468418  0.321383  0.408956  1.237482   
3  0.829697  0.614508  26.507046  0.412047  0.279256  0.471479  1.316700   
4  0.811454  0.487067  25.263819  0.448788  0.258597  0.439043  1.419255   

         PF  GAMES_PLAYED  LEAGUE_LE

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Metric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib  # To save and load the scaler
from tensorflow.keras.backend import round as kround


def prepare_dataset(game_logs_file, features_file):
    """
    Prepare dataset for training using team-specific features.

    Args:
        game_logs_file: CSV file containing game logs with TEAM1, TEAM2, and WL columns.
        features_file: CSV file to save aggregated team features.

    Returns:
        X: Feature matrix for training.
        y: Target vector (win/loss).
        feature_columns: List of feature names used in the dataset.
    """
    # Load game logs
    game_logs = pd.read_csv(game_logs_file)

    # Aggregate features by team
    team_features = (
        game_logs.groupby("TEAM1").mean(numeric_only=True).reset_index().rename(columns={"TEAM1": "TEAM"})
    )
    team_features.to_csv(features_file, index=False)

    # Merge aggregated features for TEAM1 and TEAM2
    game_logs = game_logs.merge(
        team_features.add_suffix("_TEAM1"), left_on="TEAM1", right_on="TEAM_TEAM1"
    ).merge(
        team_features.add_suffix("_TEAM2"), left_on="TEAM2", right_on="TEAM_TEAM2"
    )

    # Drop unnecessary columns
    columns_to_drop = ["TEAM_TEAM1", "TEAM_TEAM2"]
    game_logs.drop(columns=[col for col in columns_to_drop if col in game_logs.columns], inplace=True)

    # Create feature differences and ratios
    numeric_columns = [col for col in game_logs.columns if col.endswith("_TEAM1")]
    for col in numeric_columns:
        base_col = col.replace("_TEAM1", "")
        game_logs[f"{base_col}_DIFF"] = game_logs[f"{base_col}_TEAM1"] - game_logs[f"{base_col}_TEAM2"]
        game_logs[f"{base_col}_RATIO"] = game_logs[f"{base_col}_TEAM1"] / (game_logs[f"{base_col}_TEAM2"] + 1e-5)

    # Handle missing values
    game_logs.fillna(0, inplace=True)

    # Extract features and target
    feature_columns = game_logs.select_dtypes(include=np.number).columns.difference(["WL"])

    # Print the feature names
    print("Feature Columns Used in the Model:")
    print(feature_columns.tolist())

    X = game_logs[feature_columns].to_numpy()
    y = game_logs["WL"].astype(int).to_numpy()

    return X, y, feature_columns


import tensorflow.keras.backend as K

def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy metric to evaluate the model based on given conditions.
    If the prediction is in [0, 0.5) and true label is 0, it's correct.
    If the prediction is in [0.5, 1] and true label is 1, it's correct.
    """
    condition_1 = K.cast(y_pred < 0.5, dtype="float32") * K.cast(y_true == 0, dtype="float32")
    condition_2 = K.cast(y_pred >= 0.5, dtype="float32") * K.cast(y_true == 1, dtype="float32")
    return K.mean(condition_1 + condition_2)


def build_neural_network(input_shape):
    """
    Build a neural network model.

    Args:
        input_shape: Number of input features.

    Returns:
        model: Compiled neural network model.
    """
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy", custom_accuracy])

    return model


def train_model(X, y):
    """
    Train a neural network model.

    Args:
        X: Feature matrix for training.
        y: Target vector (win/loss).

    Returns:
        model: Trained neural network model.
        scaler: Fitted scaler for feature normalization.
    """
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Handle class imbalance
    X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X_scaled, y)

    # Build and train the model
    model = build_neural_network(X_resampled.shape[1])
    model.fit(X_resampled, y_resampled, epochs=16, batch_size=32, validation_split=0.2)

    return model, scaler


def save_model_and_scaler(model, scaler, model_path="model.h5", scaler_path="scaler.pkl"):
    """
    Save trained model and scaler.
    """
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    print(f"Model saved to {model_path}, Scaler saved to {scaler_path}")


def save_feature_names(feature_names, feature_names_file="feature_names.pkl"):
    """
    Save feature names for later use during prediction.
    """
    joblib.dump(feature_names, feature_names_file)
    print(f"Feature names saved to {feature_names_file}")


def main():
    game_logs_file = "preprocessed_nba_game_logs.csv"
    features_file = "features.csv"
    model_save_path = "model.h5"
    scaler_save_path = "scaler.pkl"

    # Prepare the dataset
    X, y, feature_columns = prepare_dataset(game_logs_file, features_file)

    if X.size == 0 or y.size == 0:
        print("No data available to train the model.")
        return

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train neural network
    model, scaler = train_model(X_train, y_train)

    # Save the model and scaler
    save_model_and_scaler(model, scaler, model_save_path, scaler_save_path)

    # Save feature names after preparing the dataset
    save_feature_names(feature_columns.tolist())

    # Evaluate the model
    X_test_scaled = scaler.transform(X_test)
    test_loss, test_accuracy, test_custom_accuracy = model.evaluate(X_test_scaled, y_test)
    print(f"Neural Network - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Custom Accuracy: {test_custom_accuracy}")


if __name__ == "__main__":
    main()


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
import numpy as np
import pandas as pd
from nba_api.stats.static import teams
import joblib
from tensorflow.keras.models import load_model


def load_feature_names(feature_names_file="feature_names.pkl"):
    """
    Load saved feature names for feature alignment during prediction.
    """
    feature_names = joblib.load(feature_names_file)

    # Print feature names
    print("Feature Names Used in the Model:")
    print(feature_names)

    return feature_names


def get_team_id_by_abbreviation(team_abbreviation):
    """Retrieve the team ID by abbreviation."""
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team["abbreviation"].lower() == team_abbreviation.lower():
            return team["id"]
    raise ValueError(f"Team '{team_abbreviation}' not found! Please enter a valid abbreviation.")


def fetch_team_features(team_abbreviation, features_file, feature_names):
    """
    Retrieve the team-specific features for the given team abbreviation.

    Args:
        team_abbreviation: Abbreviation of the NBA team (e.g., 'LAL').
        features_file: CSV file containing the aggregated team features.
        feature_names: List of features expected by the model.

    Returns:
        numpy array of the team's features.
    """
    team_features = pd.read_csv(features_file)
    team_row = team_features[team_features["TEAM"] == team_abbreviation.upper()]

    if team_row.empty:
        raise ValueError(f"Features for team '{team_abbreviation}' not found in {features_file}.")

    # Ensure only numeric features are returned
    numeric_features = team_row[feature_names].apply(pd.to_numeric, errors='coerce')

    return numeric_features.to_numpy().flatten()


def predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file, model_path="model.h5", scaler_path="scaler.pkl"):
    """
    Predict the win probability for Team 1 in a matchup against Team 2.

    Args:
        team1_abbreviation: Abbreviation of Team 1 (e.g., 'LAL').
        team2_abbreviation: Abbreviation of Team 2 (e.g., 'BOS').
        features_file: CSV file containing aggregated team features.
        model_path: Path to the trained neural network model file.
        scaler_path: Path to the scaler file for feature normalization.
    """
    # Load model, scaler, and feature names
    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    feature_names = load_feature_names()

    # Fetch features for both teams
    team1_features = fetch_team_features(team1_abbreviation, features_file, feature_names)
    team2_features = fetch_team_features(team2_abbreviation, features_file, feature_names)

    # Create matchup feature differences and ratios
    matchup_features = np.concatenate([
        team1_features - team2_features,
        team1_features / (team2_features + 1e-5)  # Avoid division by zero
    ]).reshape(1, -1)

    # Scale the matchup features
    matchup_features_scaled = scaler.transform(matchup_features)

    # Predict win probability for Team 1
    win_probability = model.predict(matchup_features_scaled)[0][0]

    print(f"\nWin Probability for {team1_abbreviation} vs {team2_abbreviation}: {win_probability * 100:.2f}%")


def display_team_data():
    """
    Display available team abbreviations and names for user reference.
    """
    nba_teams = teams.get_teams()
    print("Available NBA Teams:")
    for team in nba_teams:
        print(f"{team['abbreviation']} - {team['full_name']}")


def main():
    """Main function to handle user input and prediction."""
    features_file = "features.csv"  # Path to the features file

    # Display team data before taking user input
    display_team_data()

    team1_abbreviation = input("Enter Team 1 abbreviation (e.g., 'LAL' for Los Angeles Lakers): ").strip()
    team2_abbreviation = input("Enter Team 2 abbreviation (e.g., 'BOS' for Boston Celtics): ").strip()

    try:
        predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file)
    except ValueError as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()


Available NBA Teams:
ATL - Atlanta Hawks
BOS - Boston Celtics
CLE - Cleveland Cavaliers
NOP - New Orleans Pelicans
CHI - Chicago Bulls
DAL - Dallas Mavericks
DEN - Denver Nuggets
GSW - Golden State Warriors
HOU - Houston Rockets
LAC - Los Angeles Clippers
LAL - Los Angeles Lakers
MIA - Miami Heat
MIL - Milwaukee Bucks
MIN - Minnesota Timberwolves
BKN - Brooklyn Nets
NYK - New York Knicks
ORL - Orlando Magic
IND - Indiana Pacers
PHI - Philadelphia 76ers
PHX - Phoenix Suns
POR - Portland Trail Blazers
SAC - Sacramento Kings
SAS - San Antonio Spurs
OKC - Oklahoma City Thunder
TOR - Toronto Raptors
UTA - Utah Jazz
MEM - Memphis Grizzlies
WAS - Washington Wizards
DET - Detroit Pistons
CHA - Charlotte Hornets
Enter Team 1 abbreviation (e.g., 'LAL' for Los Angeles Lakers): POR
Enter Team 2 abbreviation (e.g., 'BOS' for Boston Celtics): SAS


Feature Names Used in the Model:
['AST', 'AST_DIFF', 'AST_RANK', 'AST_RANK_DIFF', 'AST_RANK_RATIO', 'AST_RANK_TEAM1', 'AST_RANK_TEAM2', 'AST_RATIO', 'AST_TEAM1', 'AST_TEAM2', 'BLK', 'BLKA', 'BLKA_DIFF', 'BLKA_RANK', 'BLKA_RANK_DIFF', 'BLKA_RANK_RATIO', 'BLKA_RANK_TEAM1', 'BLKA_RANK_TEAM2', 'BLKA_RATIO', 'BLKA_TEAM1', 'BLKA_TEAM2', 'BLK_DIFF', 'BLK_RANK', 'BLK_RANK_DIFF', 'BLK_RANK_RATIO', 'BLK_RANK_TEAM1', 'BLK_RANK_TEAM2', 'BLK_RATIO', 'BLK_TEAM1', 'BLK_TEAM2', 'DREB', 'DREB_DIFF', 'DREB_RANK', 'DREB_RANK_DIFF', 'DREB_RANK_RATIO', 'DREB_RANK_TEAM1', 'DREB_RANK_TEAM2', 'DREB_RATIO', 'DREB_TEAM1', 'DREB_TEAM2', 'FG3A', 'FG3A_DIFF', 'FG3A_RANK', 'FG3A_RANK_DIFF', 'FG3A_RANK_RATIO', 'FG3A_RANK_TEAM1', 'FG3A_RANK_TEAM2', 'FG3A_RATIO', 'FG3A_TEAM1', 'FG3A_TEAM2', 'FG3M', 'FG3M_DIFF', 'FG3M_RANK', 'FG3M_RANK_DIFF', 'FG3M_RANK_RATIO', 'FG3M_RANK_TEAM1', 'FG3M_RANK_TEAM2', 'FG3M_RATIO', 'FG3M_TEAM1', 'FG3M_TEAM2', 'FG3_PCT', 'FG3_PCT_DIFF', 'FG3_PCT_RANK', 'FG3_PCT_RANK_DIFF', 'FG3_PCT_RANK_RA

KeyError: "['AST_DIFF', 'AST_RANK_DIFF', 'AST_RANK_RATIO', 'AST_RANK_TEAM1', 'AST_RANK_TEAM2', 'AST_RATIO', 'AST_TEAM1', 'AST_TEAM2', 'BLKA_DIFF', 'BLKA_RANK_DIFF', 'BLKA_RANK_RATIO', 'BLKA_RANK_TEAM1', 'BLKA_RANK_TEAM2', 'BLKA_RATIO', 'BLKA_TEAM1', 'BLKA_TEAM2', 'BLK_DIFF', 'BLK_RANK_DIFF', 'BLK_RANK_RATIO', 'BLK_RANK_TEAM1', 'BLK_RANK_TEAM2', 'BLK_RATIO', 'BLK_TEAM1', 'BLK_TEAM2', 'DREB_DIFF', 'DREB_RANK_DIFF', 'DREB_RANK_RATIO', 'DREB_RANK_TEAM1', 'DREB_RANK_TEAM2', 'DREB_RATIO', 'DREB_TEAM1', 'DREB_TEAM2', 'FG3A_DIFF', 'FG3A_RANK_DIFF', 'FG3A_RANK_RATIO', 'FG3A_RANK_TEAM1', 'FG3A_RANK_TEAM2', 'FG3A_RATIO', 'FG3A_TEAM1', 'FG3A_TEAM2', 'FG3M_DIFF', 'FG3M_RANK_DIFF', 'FG3M_RANK_RATIO', 'FG3M_RANK_TEAM1', 'FG3M_RANK_TEAM2', 'FG3M_RATIO', 'FG3M_TEAM1', 'FG3M_TEAM2', 'FG3_PCT_DIFF', 'FG3_PCT_RANK_DIFF', 'FG3_PCT_RANK_RATIO', 'FG3_PCT_RANK_TEAM1', 'FG3_PCT_RANK_TEAM2', 'FG3_PCT_RATIO', 'FG3_PCT_TEAM1', 'FG3_PCT_TEAM2', 'FGA_DIFF', 'FGA_RANK_DIFF', 'FGA_RANK_RATIO', 'FGA_RANK_TEAM1', 'FGA_